## Aggregating results to DataFrame

In [2]:
import os
import lib
import numpy as np
import pandas as pd
import json
from pathlib import Path

base_path = json.load(open("secrets.json", "r"))["Experiment_Folder"]

method2exp = {
    "real": "adult/20_12_2022-REAL-BASELINE/outputs/exp/adult/ddpm_real/final_eval/",
    "tab-ddpm": "adult/21_12_2022-identity-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_identity_best/final_eval/",
    "tab-ddpm-simTune": "adult/12_02_2023-identity_sim_tune-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_identity_sim_tune_best/final_eval/",
    "tab-ddpm-bgm": "adult/20_12_2022-bgm-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_bgm_best/final_eval/",
    "tab-ddpm-bgm-simTune" : "adult/12_02_2023-bgm_sim_tune-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_bgm_sim_tune_best/final_eval/",
    "tab-ddpm-ft" : "adult/08_02_2023-ft-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_ft_best/final_eval/",
    "smote": "adult/02_01_2023-identity-SMOTE/outputs/exp/adult/smote/final_eval/",
    "ctabgan+": "adult/05_01_2023-identity-CTABGAN-Plus/outputs/exp/adult/ctabgan-plus/final_eval/",
    "ctabgan": "adult/02_01_2023-identity-CTABGAN/outputs/exp/adult/ctabgan/final_eval/",
    "tvae": "adult/03_01_2023-identity-TVAE/outputs/exp/adult/tvae/final_eval/",
} 
for k,v in method2exp.items():
    method2exp[k] = Path(os.path.join(base_path, v))

eval_file = "eval_catboost.json"
sim_file = "results_similarity.json"
eval_sim = "eval_similarity.json"
show_std = False
columns = ["method"] 
# df = pd.DataFrame(columns=["method"] + [_[:3].upper() for _ in DATASETS])
df = []
for algo in method2exp: 
    base_df = pd.DataFrame([algo], columns=["method"])
    metric_df = pd.DataFrame()
    metrics=["acc","f1","roc_auc"]

    if not os.path.exists(method2exp[algo] / eval_file):
        print(f"File {eval_file} not found for {algo}")
        metric_df = pd.DataFrame([["---"]*len(metrics)], columns=metrics)

    else:
        res_dict = lib.load_json(method2exp[algo] / eval_file)
        for metric in metrics:
            if algo == "real":
                res = f'{res_dict["real"]["test"][metric + "-mean"]:.4f}' 
                if show_std: res += f'+-{res_dict["real"]["test"][metric + "-std"]:.4f}'
                metric_df[metric] = [res]
            else:
                res = f'{res_dict["synthetic"]["test"][metric + "-mean"]:.4f}'
                if show_std: res += f'+-{res_dict["synthetic"]["test"][metric + "-std"]:.4f}'
                metric_df[metric] = [res]

    sim_metrics = ["score", "basic_score","corr_score", "ml_score","sup_score", "pmse_score"]

    if os.path.exists(method2exp[algo] / eval_sim):
        sim_res_dict = lib.load_json(method2exp[algo] / eval_sim)
        tmp = {}
        for k,v in sim_res_dict.items():
            if "count" in k: continue
            if "std" in k and show_std: 
                if show_std:
                    v = f'+-{float(v):.4f}'
                continue
            # if "mean" in k remove "-mean" from key
            if "mean" in k: 
                k = k.replace("-mean", "")
                v = f'{float(v):.4f}'
            tmp[k] = v 	    
            
        sim_df = pd.DataFrame([tmp], columns=sim_metrics)

    else:
        sim_res_dict = lib.load_json(method2exp[algo] / sim_file)
        sim_df = pd.DataFrame([sim_res_dict["sim_score"]], columns=sim_metrics)
    # rename score to sim_score
    sim_df = sim_df.rename(columns={"score": "sim_score"})    
    # format all floats to :.4f
    sim_df = sim_df.applymap(lambda x: f'{x:.4f}' if isinstance(x, float) else x)

    base_df = pd.concat([base_df, metric_df, sim_df], axis=1)
    
    df.append(base_df)


calculate_diff = False
results = pd.concat(df, axis=0)
metrics = results.columns.tolist()
metrics.remove("method")
cols= ["method"]
if calculate_diff:
    for metric in metrics:
        results[metric] = pd.to_numeric(results[metric])
        results[f"{metric}-diff"] = results[metric] - results.loc[results["method"] == "real", metric].values[0]
        cols.append(metric)
        cols.append(f"{metric}-diff")
    results=results.reindex(cols, axis=1)

results.reset_index(drop=True, inplace=True)
results

,method,acc,f1,roc_auc,sim_score,basic_score,corr_score,ml_score,sup_score,pmse_score
0,real,0.8742,0.8152,0.9276,0.9598,0.9922,0.9433,0.9975,0.9839,0.8820
1,tab-ddpm,0.8598,0.7941,0.9128,0.7586,0.9730,0.9189,0.9923,0.8741,0.0349
2,tab-ddpm-simTune,0.8556,0.7823,0.9078,0.8520,0.9764,0.9210,0.9910,0.9522,0.4196
3,tab-ddpm-bgm,0.8632,0.7985,0.9165,0.7418,0.9642,0.9183,0.9955,0.8307,0.0004
4,tab-ddpm-bgm-simTune,0.8586,0.7917,0.9109,0.8567,0.9823,0.8579,0.9913,0.9197,0.5323
5,tab-ddpm-ft,0.7849,0.5516,0.8212,0.5951,0.4950,0.6482,0.8691,0.9633,0.0000
6,smote,0.8582,0.7912,0.9104,0.7228,0.9528,0.8651,0.9925,0.8038,0.0000
7,ctabgan+,0.8547,0.7747,0.9070,0.7503,0.9692,0.8818,0.9902,0.8915,0.0186
8,ctabgan,0.8499,0.7750,0.8995,0.7406,0.9397,0.8321,0.9845,0.9468,0.0000
9,tvae,0.8450,0.7805,0.9003,0.6575,0.8544,0.8139,0.9620,0.6572,0.0000


In [ ]:
# DATASETS = [
#     "abalone",
#     "adult",
#     "buddy",
#     "california",
#     "cardio",
#     "churn2",
#     "default",
#     "diabetes",
#     "fb-comments",
#     "gesture",
#     "higgs-small",
#     "house",
#     "insurance",
#     "king",
#     "miniboone",
#     "wilt"
# ]

# _REGRESSION = [
#     "abalone",
#     "california",
#     "fb-comments",
#     "house",
#     "insurance",
#     "king",
# ]

# method2exp = {
#     "real": "adult/20_12_2022-REAL-BASELINE/outputs/adult/ddpm_real/final_eval/",
#     "tab-ddpm-bgm": "adult/20_12_2022-bgm-50optuna-ts26048-catboost-tune-CatboostAndSimilarityEval-syntheticEval/outputs/exp/adult/ddpm_bgm_best/final_eval/",
#     # "tab-ddpm": "exp/{}/ddpm_cb_best/",
#     # "smote": "exp/{}/smote/",
#     # "ctabgan+": "exp/{}/ctabgan-plus/",
#     # "ctabgan": "exp/{}/ctabgan/",
#     # "tvae": "exp/{}/tvae/"
# }

# eval_file = "eval_catboost.json"
# sim_file = "results_similarity.json"
# show_std = False
# df = pd.DataFrame(columns=["method"] + [_[:3].upper() for _ in DATASETS])

# for algo in method2exp: 
#     algo_res = []
#     for ds in DATASETS:
#         if not os.path.exists(os.path.join(method2exp[algo].format(ds), eval_file)):
#             algo_res.append("--")
#             continue
#         metric = "r2" if ds in _REGRESSION else "f1"
#         res_dict = lib.load_json(os.path.join(method2exp[algo].format(ds), eval_file))

#         if algo == "real":
#             res = f'{res_dict["real"]["test"][metric + "-mean"]:.4f}' 
#             if show_std: res += f'+-{res_dict["real"]["test"][metric + "-std"]:.4f}'
#         else:
#             res = f'{res_dict["synthetic"]["test"][metric + "-mean"]:.4f}'
#             if show_std: res += f'+-{res_dict["synthetic"]["test"][metric + "-std"]:.4f}'

#         algo_res.append(res)
#     df.loc[len(df)] = [algo] + algo_res